# 6.804 Final Project

Meia Alsup, Sravya Bhamidipati, Anelise Newman

In [1]:
from string import digits
import json
import pickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from random import randrange

from IPython.display import Image, display
from IPython.core.display import HTML 
import urllib.request as request

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import wordnet
import nltk


In [2]:
 #ocr = pkl.load(open('raw_ocr_output.pickle','rb'))
#less_ocr = {key: ocr[key] for i, key in enumerate(ocr) if i < 100}
#less_ocr

In [3]:
#f = open('icon_annotations_testing.pickle','rb')
#icons = pkl.load(f)
#f.close()

## Clean input to just English words



In [4]:
f = open('google_text_extraction_output.pckl','rb')
infographic_name_to_words = pkl.load(f)
f.close()

print('We have extracted words from %d infographics.'%len(infographic_name_to_words))

We have extracted words from 62361 infographics.


In [5]:
def contains_english(set_of_words):
    english = set(['the', 'she', 'different', 'time', 'and', 'client', 'sports', 'grammar', 'family', 'report'])
    spanish = set(['el', 'es', 'cuando', 'esta', 'malo', 'que', 'tiene', 'para'])
    if bool(set(set_of_words) & spanish):
        return False
    if bool(set(set_of_words) & english):
        return True
    return False

In [6]:
english  = {key: val for key, val in infographic_name_to_words.items() if contains_english(val)}
len(english)

46952

In [7]:
json.dump(english, open('filtered.json', 'w'))
json.dump([key for key, val in english.items()], open('filtered_keys.json', 'w'))

In [10]:
newEnglish = {}
newList = []
stemmer = PorterStemmer()
for url, wordList in english.items():
    thisList = []
    for word in wordList:
        if len(word) > 2 and word.isalpha() and word not in STOPWORDS and wordnet.synsets(word):
            thisList.append(word.lower())
            #thisList.append(stemmer.stem(WordNetLemmatizer().lemmatize(word.lower(), pos='v')))
    newList.append(thisList)
    newEnglish[url] = thisList

In [ ]:
NUM_TOPICS = 20

In [20]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(newList)
common_dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=10000)

common_corpus = [common_dictionary.doc2bow(text) for text in newList]
# Train the model on the corpus.
id2token = {val:key for key, val in common_dictionary.token2id.items()}

In [ ]:
# Train the model on the corpus.
#lda = LdaModel(common_corpus, num_topics=NUM_TOPICS)

In [ ]:
#for i in range(NUM_TOPICS):
#    terms = lda.get_topic_terms(i, 20)
#    print(f'Topic number {i}')
#    val = {id2token[t[0]]: t[1] for t in terms}
#    #print(f'Top 20 terms: {val}')
#    print(f'Top 20 terms: {val.keys()}')

In [43]:
#saved = lda.save('saved_lda_15_topics')

## Intersction with infographics that have labeled bounding boxes

In [11]:
# annotations from task where participants marked bounding boxes around all icons on an infographic
icon_dict = pkl.load(open( "icon_annotations_all.pickle", "rb") )
icons_english  = set(icon_dict.keys())

In [12]:
base='http://thumbnails.visually.netdna-cdn.com'

In [13]:
intersection = set([url_ for url_ in english.keys() if url_ in icons_english])

In [14]:
len(intersection)

1079

In [ ]:
for url_ in intersection:
    #display(Image(url=f'{base}/{url_}'))
    request.urlretrieve(f'{base}/{url_}', f'images/{url_}')

## See category labels for chosen inforgraphics


In [17]:
lda = LdaModel.load('ldas/saved_lda_20_topics')

In [18]:
categories = {
    0: 'healthcare', # combine
    1: 'economy and government',
    2: 'work and positivity',  #gon3
    3: 'celebrations and family',
    4: 'Internet and social media',
    5: 'international world',
    6: 'body and health',  #combine into 0
    7: 'shopping and travel',
    8: 'online shopping',
    9: 'marketing',
    10: 'beach',  # gone
    11: 'education',
    12: 'mobile phones and devices',
    13: 'food',
    14: 'cars and driving',
    15: 'culture and colors',
    16: 'money and finances',
    17: 'energy and environment',
    18: 'house and home',
    19: 'business and technology',
}
category_to_id = {v: k for k, v in categories.items()}

In [41]:
infographics = json.load(open('study_infographics.json'))
infographic_id_to_url_map = {i: entry['url'] for i , entry in enumerate(infographics)}
url_to_id_map = {v:k for k,v in infographic_id_to_url_map.items()}
json.dump(infographic_id_to_url_map, open('infographic_id_to_url_map.json', 'w'))
json.dump(url_to_id_map, open('infographic_url_to_id_map.json', 'w'))

In [43]:
url_to_id_map


{'the-facts-about-heart-disease_50290aaf3b5fd.jpg': 0,
 'the-egreen-society-how-britain-can-recycle-and-be-more-responsible_535a49b55ca1f.jpg': 1,
 'super-dad_502910d3ad406.png': 2,
 'hunters-vs-farmers-sales-personas_51e5924c2f823.png': 3,
 'calories-in-wine-vs-beer_50982977dc9eb.jpg': 4,
 'interesting-facts-on-car-crashes_509c3e82230cf.jpg': 5,
 'telecom-present--the-future_530b60f629fab.jpg': 6,
 'anatomy-of-a-wedding-gown_51c3402050877.jpg': 7,
 '31-days-of-refreshingly-chilled-lemonade_5029153eb8c76.png': 8,
 'why-india-cant-feed-her-people_50290b996c340.jpg': 9,
 'texting-while-driving-statistics--facts_50290ab6860e7.png': 10,
 'a-geographical-trap-landlocked-developing-countries_50951a6dacde9.jpg': 11}

In [57]:
confidences = {}
for url in os.listdir('test/final'):
    
    #display(Image(url=f'{base}/{url}', width=300))
    confidence = {c: prob for c, prob in lda[common_dictionary.doc2bow(newEnglish[url])] if c not in [2,10]}
    confidence[0] = confidence.get(0, 0) + confidence.get(6, 0)
    if 6 in confidence:
        del confidence[6]
    top = sorted(confidence.items(), key=lambda x: (x[1]), reverse=True)[:5]
    confidences[url_to_id_map[url]] = {key[0]: str(confidence[key[0]]) for key in top}
json.dump(confidences, open('text_distribution.json', 'w'))

In [90]:
image_categories = []
for url in os.listdir('test/final'):
    
    #display(Image(url=f'{base}/{url}', width=300))
    confidence = {}
    for c, prob in lda[common_dictionary.doc2bow(newEnglish[url])]:
        confidence[categories[c]] = prob
    confidence = sorted(confidence.items(), key=lambda x: (x[1],x[0]), reverse=True)
    #print(url, confidence)
    image_categories.append({
        "categories": [
            [category_to_id[cat[0]],
             cat[0]]
            for cat in confidence[:5] if category_to_id[cat[0]] not in [2, 6, 10]
        ],
        "url": url
    })
image_categories

[{'categories': [[0, 'healthcare'], [18, 'house and home'], [13, 'food']],
  'url': 'the-facts-about-heart-disease_50290aaf3b5fd.jpg'},
 {'categories': [[0, 'healthcare'],
   [3, 'celebrations and family'],
   [1, 'economy and government'],
   [13, 'food']],
  'url': 'super-dad_502910d3ad406.png'},
 {'categories': [[19, 'business and technology'],
   [7, 'shopping and travel'],
   [8, 'online shopping'],
   [9, 'marketing']],
  'url': 'hunters-vs-farmers-sales-personas_51e5924c2f823.png'},
 {'categories': [[13, 'food'],
   [3, 'celebrations and family'],
   [0, 'healthcare']],
  'url': 'calories-in-wine-vs-beer_50982977dc9eb.jpg'},
 {'categories': [[19, 'business and technology'],
   [12, 'mobile phones and devices'],
   [5, 'international world'],
   [8, 'online shopping'],
   [7, 'shopping and travel']],
  'url': 'telecom-present--the-future_530b60f629fab.jpg'},
 {'categories': [[1, 'economy and government'],
   [17, 'energy and environment'],
   [7, 'shopping and travel'],
   [14, '

In [97]:
cleaned = []
for entry in image_categories:
    clean_categories = []
    for category in entry['categories']:
        if category[0] == 0:
            clean_categories.append([0, 'health'])
        else:
            clean_categories.append(category)
    while len(clean_categories) < 5:
        cur_categories = [x[0] for x in clean_categories]
        i = 2
        while i in [2, 6, 10] or i in cur_categories:
            i = randrange(1, 19)
        clean_categories.append([i, categories[i]])    
        
    cleaned.append({
        'categories': clean_categories,
        'url': entry['url']
    }
cleaned

[{'categories': [[0, 'health'],
   [18, 'house and home'],
   [13, 'food'],
   [7, 'shopping and travel'],
   [9, 'marketing']],
  'url': 'the-facts-about-heart-disease_50290aaf3b5fd.jpg'},
 {'categories': [[0, 'health'],
   [3, 'celebrations and family'],
   [1, 'economy and government'],
   [13, 'food'],
   [17, 'energy and environment']],
  'url': 'super-dad_502910d3ad406.png'},
 {'categories': [[19, 'business and technology'],
   [7, 'shopping and travel'],
   [8, 'online shopping'],
   [9, 'marketing'],
   [5, 'international world']],
  'url': 'hunters-vs-farmers-sales-personas_51e5924c2f823.png'},
 {'categories': [[13, 'food'],
   [3, 'celebrations and family'],
   [0, 'health'],
   [14, 'cars and driving'],
   [7, 'shopping and travel']],
  'url': 'calories-in-wine-vs-beer_50982977dc9eb.jpg'},
 {'categories': [[19, 'business and technology'],
   [12, 'mobile phones and devices'],
   [5, 'international world'],
   [8, 'online shopping'],
   [7, 'shopping and travel']],
  'url': '